In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

### 1.1 Reading data from a csv file

The `csv` format ([comma-separated values](https://en.wikipedia.org/wiki/Comma-separated_values)) is the most common file-based format to exchange data. The format is not standardized, and you will encounter many varieties in practice - for instance, how values are separated (not always by comma!), how `string` values are represented, or how dates are represented. It is quite important to know how you can translate these different formats to workable standard.

`pandas` can read data from a `csv` file using the `read_csv` function with numerous options to parse the file contents.

We will be using two `csv` files  available [here](http://donnees.ville.montreal.qc.ca/dataset/velos-comptage) (in French) containing data on 
1. the usage of 7 different bike paths in Montréal for each day during 2015 available, and 
2. the location of the stations that measured usage.

Let's first look at version of the usage data from 2012 that required some parsing. It is included in the 'data' subfolder of this repository.

In [3]:
bikes_2012 = pd.read_csv('../data/bike_usage_2012.csv')
bikes_2012.head(3)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 15: invalid continuation byte

In [ ]:
!file -I ../data/bike_usage_2012.csv

There are at least two issues: 
- the cells are not separated by commas but by semicolons, and
- the `string` representation is off where there seem to be 'special' 
characters (french accents).

We'll use some of the `read_csv` options that to properly parse the file:

* `sep`: change the column separator to a `;`
* `encoding`: set to `'iso-8859-1'` (or `'latin1'` - the default is `'utf8'` - background [here](http://www.joelonsoftware.com/articles/Unicode.html))
* `parse_dates`: Parse 'Date' column by indicating that our dates have day instead of month first
* `index_col`: set the index to be the parsed 'Date' column

In [ ]:
bikes_2012_parsed = pd.read_csv('../data/bike_usage_2012.csv', 
                                sep=';', 
                                encoding='iso-8859-1', 
                                parse_dates=['Date'], 
                                dayfirst=True, 
                                index_col='Date')
bikes_2012_parsed.head(3)

`read_csv` can read from a file on disk or from a url. The data are included in the 'data' subfolder of this repo, but let's use the urls:

In [ ]:
bike_usage_url = 'http://donnees.ville.montreal.qc.ca/dataset/f170fecc-18db-44bc-b4fe-5b0b6d2c7297/resource/64c26fd3-0bdf-45f8-92c6-715a9c852a7b/download/comptagesvelo2012.csv'
bike_station_url = 'http://donnees.ville.montreal.qc.ca/dataset/f170fecc-18db-44bc-b4fe-5b0b6d2c7297/resource/c7d0546a-a218-479e-bc9f-ce8f13ca972c/download/localisationcompteursvelo2015.csv'

In [ ]:
bike_usage = pd.read_csv(bike_usage_url, index_col='Date')
bike_usage.head()

Let's get rid of the 'Unnamed' column (the file as the `hour` and `minute` part of the date in a separate column with not header, but we don't need this for daily data).

In [ ]:
# dropping the first COLUMN, so need to use axis=1
# using inplace, so the original object is modified 
# so you'll get an error if you run this twice!
bike_usage.drop('Unnamed: 1', axis=1, inplace=True)
bike_usage.info()

In [ ]:
bike_usage.head()

In [ ]:
stations = pd.read_csv(bike_station_url, encoding='latin1')
stations.nom_comptage.sort_values().reset_index(drop=True)

### Creating a Map of Bike Usage

In [ ]:
import folium
map_center = stations[['coord_Y', 'coord_X']].mean().tolist()
map = folium.Map(location=map_center)
map

In [ ]:
avg_bike_usage = bike_usage.mean().to_frame('avg_usage')
avg_bike_usage.sort_index()

In [ ]:
stations_summary = stations.merge(avg_bike_usage, 
                left_on='nom_comptage', right_index=True, how='left').loc[:,
                  ['nom', 'nom_comptage', 'coord_X', 'coord_Y', 'avg_usage']]
stations_summary.info()

In [ ]:
missing_stations = stations[~stations.nom_comptage.isin(avg_bike_usage.index.tolist())].nom_comptage
print missing_stations

In [ ]:
# alternative to achieve the same result
missing_stations = stations_summary[pd.isnull(stations_summary.avg_usage)].nom_comptage
{m:m.encode('ascii','ignore') for m in missing_stations}

In [ ]:
# stations from 'usage' data that don't match
to_match = avg_bike_usage[~avg_bike_usage.index.isin(stations_summary.nom_comptage)].index.tolist()
{m:m.decode('ascii','ignore') for m in to_match}

In [ ]:
# Testing string matching for all missing stations
from difflib import SequenceMatcher
missing_station_list = [m.encode('ascii','ignore') for m in missing_stations]
to_match_list = [m.decode('ascii','ignore') for m in to_match]
for missing_station in missing_station_list:
    print pd.Series({station: SequenceMatcher(None, 
          missing_station,station).ratio()*100 
                     for station in to_match_list})\
    .sort_values(ascending=False).to_frame(missing_station)        

In [ ]:
# since the above works, let's 
def get_match(missing_station):
    matches = {station: SequenceMatcher(None, 
          missing_station,station).ratio()
                     for station in to_match}
    return max(matches, key=matches.get)

matches = pd.concat([missing_stations, missing_stations.apply(lambda x: get_match(x))], axis=1)
matches.columns = ['missing', 'replacement']
matches

In [ ]:
match_dict = dict(zip(matches.missing, matches.replacement))
stations.nom_comptage = stations.nom_comptage.apply(lambda x: match_dict.get(x, x))

In [ ]:
stations.nom_comptage

In [ ]:
stations_summary = stations.merge(avg_bike_usage, 
                left_on='nom_comptage', right_index=True, how='left').loc[:,
                  ['nom', 'nom_comptage', 'coord_X', 'coord_Y', 'avg_usage']]
stations_summary.info()

<style>
    @font-face {
        font-family: "Computer Modern";
        src: url('http://mirrors.ctan.org/fonts/cm-unicode/fonts/otf/cmunss.otf');
    }
    div.cell{
        width:800px;
        margin-left:16% !important;
        margin-right:auto;
    }
    h1 {
        font-family: Helvetica, serif;
    }
    h4{
        margin-top:12px;
        margin-bottom: 3px;
       }
    div.text_cell_render{
        font-family: Computer Modern, "Helvetica Neue", Arial, Helvetica, Geneva, sans-serif;
        line-height: 145%;
        font-size: 130%;
        width:800px;
        margin-left:auto;
        margin-right:auto;
    }
    .CodeMirror{
            font-family: "Source Code Pro", source-code-pro,Consolas, monospace;
    }
    .text_cell_render h5 {
        font-weight: 300;
        font-size: 22pt;
        color: #4057A1;
        font-style: italic;
        margin-bottom: .5em;
        margin-top: 0.5em;
        display: block;
    }
    
    .warning{
        color: rgb( 240, 20, 20 )
        }  

In [ ]:
import folium
map_center = stations[['coord_Y', 'coord_X']].mean().tolist()
map = folium.Map(location=map_center, zoom_start=13)
map

In [ ]:
for i, location in stations_summary.iterrows():
    folium.CircleMarker(location.loc[['coord_Y', 'coord_X']],
                    radius= np.sqrt(location.avg_usage/np.pi)*10,
                    popup=location.nom_comptage.decode('ascii', 'ignore'),
                    color='#3186cc',
                    fill_color='#3186cc',
                   ).add_to(map)

In [ ]:
map

In [ ]:
stations_summary.to_excel('test.xlsx')